Dataset source = http://snap.stanford.edu/data/wiki-RfA.html <br>
Dataset title = "Wikipedia Requests for Adminship (with text)"<br>
Dataset public = Yes

#### Stats coming from dataset
(median/mean: 19/34 tokens)<br>
Nodes	10,835<br>
Edges	159,388<br>
Triangles	956,428<br>
Type of graph: Directed Signed Network

#### Dataset format
*Fields*<br>
SRC: user name of source, i.e., voter<br>
TGT: user name of target, i.e., the user running for election<br>
VOT: the source's vote on the target (-1 = oppose; 0 = neutral; 1 = support)<br>
RES: the outcome of the election (-1 = target was rejected as admin; 1 = target was accepted)<br>
YEA: the year in which the election was started<br>
DAT: the date and time of this vote<br>
TXT: the comment written by the source, in wiki markup<br>
<br>
<br>
*Example*<br>
SRC:Guettarda<br>
TGT:Lord Roem<br>
VOT:1<br>
RES:1<br>
YEA:2013<br>
DAT:19:53, 25 January 2013<br>
TXT:'''Support''' per [[WP:DEAL]]: clueful, and unlikely to break Wikipedia.<br>

In [1]:
import networkx as nx
import  pickle
import pandas as pd
import numpy as np
import glob
import os
import re
from collections import Counter
import pylab as plt
from datascience import *
import csv

Generate our indeces

In [2]:
iSrc = 0
iTgt = 1
iVot = 2
iRes = 3
iTxt = 4
iDat = 5
iYea = 6
iSrcId = 7
iTgtId = 8
iSrcInDegree = 9
iSrcOutDegree = 10
iTgtInDegree = 11
iTgtOutDegree =12
iNumWords = 13

## Read the file

In [3]:
file = "./wiki-RfAv2.txt"
# Read file and save in content all the lines
with open(file, mode="r",encoding="utf-8") as f:
       content = f.read().splitlines()

# We need a list of all the node's names
# we will append into a list all the names, from src and tgt, so we can
# include all of them
srcTgtNodes = []
for text in content:
    textsp = re.split(r":::", text)
    if textsp[0] == 'TGT' or textsp[0] == 'SRC':
        srcTgtNodes.append(textsp[1])
#fullTable = np.chararray((len(srcTgtNodes), 14))
# Plust one for the column headers name
r, c = 14, len(srcTgtNodes)+1

iSrc = 0
iTgt = 1
iVot = 2
iRes = 3
iTxt = 4
iDat = 5
iYea = 6
iSrcId = 7
iTgtId = 8
iSrcInDegree = 9
iSrcOutDegree = 10
iTgtInDegree = 11
iTgtOutDegree =12
iNumWords = 13

fullTable = [[" " for x in range(r)] for y in range(c)] 
fullTable[0][0]="SRC"
fullTable[0][1]="TGT"
fullTable[0][2]="VOT"
fullTable[0][3]="RES"
fullTable[0][4]="TXT"
fullTable[0][5]="DAT"
fullTable[0][6]="YEA"
fullTable[0][7]="SRC_ID"
fullTable[0][8]="TGT_ID"
fullTable[0][9]="SRC_IN_DEGREE"
fullTable[0][10]="SRC_OUT_DEGREE"
fullTable[0][11]="TGT_IN_DEGREE"
fullTable[0][12]="TGT_OUT_DEGREE"
fullTable[0][13]="NUM_WORDS"
#Now that we have all the users, let's remove duplicates
userSet = set(srcTgtNodes)
usersDictionary = dict()
a = enumerate(userSet)
for k,v in a:
    usersDictionary[v]=k

First let's setup the directed graph instance.

In [4]:
G = nx.DiGraph()

Now let's build the graph and also create the golbal list

In [5]:
#Check if all the fields are present
DAT=""
RES=""
SRC=""
TGT=""
TXT=""
VOT=""
YEA=""
count = 0
for text in content:
    textsp = re.split(r":::", text)
    if textsp[0] == 'VOT':
        VOT=textsp[1]
    if textsp[0] == 'YEA':
        YEA=textsp[1]
    if textsp[0] == 'DAT':
        DAT=textsp[1]
    if textsp[0] == 'RES':
        RES=textsp[1]
    if textsp[0] == 'SRC':
        SRC=textsp[1]
    if textsp[0] == 'TGT':
        TGT=textsp[1]
    if textsp[0] == 'TXT':
        TXT=textsp[1]
        # Now that we have all data per row, let's create node
        G.add_edge(SRC, TGT)
        G[SRC][TGT]['VOT'] = VOT
        G[SRC][TGT]['RES'] = RES
        G[SRC][TGT]['YEA'] = YEA
        G[SRC][TGT]['DAT'] = DAT
        G[SRC][TGT]['TXT'] = TXT
        
        
        # Fill all fields for the big table
        fullTable[count+1][iSrc]=SRC
        fullTable[count+1][iTgt]=TGT
        fullTable[count+1][iVot]=VOT
        fullTable[count+1][iRes]=RES
        fullTable[count+1][iTxt]=TXT
        fullTable[count+1][iDat]=DAT
        fullTable[count+1][iYea]=YEA
        #fullTable[count][iSrcId]=
        #fullTable[count][iTgtId]=
        #fullTable[count][iSrcInDegree]=
        #fullTable[count][iSrcOutDegree]=
        #fullTable[count][iTgtInDegree]=
        #fullTable[count][iTgtOutDegree]=
        fullTable[count+1][iNumWords]=len((TXT).split())
        
        # Clean variables
        DAT=""
        RES=""
        SRC=""
        TGT=""
        TXT=""
        VOT=""
        YEA=""
        
        # Increase counter
        count = count + 1

We need to convert first the tables we will use as numeric

In [6]:
"""
iSrc = 0
iTgt = 1
iVot = 2
iRes = 3
iTxt = 4
iDat = 5
iYea = 6
iSrcId = 7
iTgtId = 8
iSrcInDegree = 9
iSrcOutDegree = 10
iTgtInDegree = 11
iTgtOutDegree =12
iNumWords = 13
"""
counter=0
for a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13 in fullTable:
    if counter != 0:    
        # Here we convert the user into its numerical representation
        if a0 != ' ':
            fullTable[counter][7] = usersDictionary[a0]
        if a1 != ' ':
            fullTable[counter][8] = usersDictionary[a1]
        # Here we fill the in-degree and out-degree
        fullTable[counter][9]=G.in_degree(a0)
        fullTable[counter][10]=G.out_degree(a0)
        fullTable[counter][11]=G.in_degree(a1)
        fullTable[counter][12]=G.out_degree(a1)
    counter=counter+1


In [7]:
with open("low_mem_data.csv","w+",encoding="utf-8") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(fullTable)